In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random

import spacy
from transformers import AutoTokenizer
import string
import re

import itertools

from tqdm import tqdm

In [2]:
df = pd.read_csv('/kaggle/input/english-to-hindi/hin.txt',sep = '\t',names = ['english_sentence','hindi_sentence'])
df.head()

,english_sentence,hindi_sentence
0,Wow!,वाह!
1,Help!,बचाओ!
2,Jump.,उछलो.
3,Jump.,कूदो.
4,Jump.,छलांग.


In [3]:
#torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
eng_tokenizer = spacy.load("en_core_web_sm")
hin_tokenizer = AutoTokenizer.from_pretrained('raunaqjabbal/hindi-tokenizer')

tokenizer_config.json:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/607k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [5]:
class Vocabulary:
    
    def __init__(self,freq_threshold):
        self.freq_threshold = freq_threshold
        
        self.en_stoi = {'<pad>':0,'<sos>':1,'<eos>':2,'<unk>':3}
        self.en_itos = {0:'<pad>',1:'<sos>',2:'<eos>',3:'<unk>'}
        
        self.hi_stoi = {'<pad>':0,'<sos>':1,'<eos>':2,'<unk>':3}
        self.hi_itos = {0:'<pad>',1:'<sos>',2:'<eos>',3:'<unk>'}
        
    def __len__(self):
        return len(self.en_stoi)
    
    def tokenize_eng(self, en_text):
        return [token.text.lower() for token in eng_tokenizer.tokenizer(en_text)]
    
    def tokenize_hin(self,hi_text):
        return hin_tokenizer.tokenize(hi_text)
    
    def build_vocabulary(self, sentences, lang):
        frequencies = {}
        idx = 4
        
        if lang == 'english':
            for sentence in sentences:
                for word in self.tokenize_eng(sentence):
                    if word not in frequencies:
                        frequencies[word] = 1
                    else:
                        frequencies[word] += 1

                    if frequencies[word] == self.freq_threshold and len(word)>1 and word.isalpha():
                        self.en_stoi[word] = idx
                        self.en_itos[idx] = word
                        idx += 1
        else:
            for sentence in sentences:
                for word in self.tokenize_hin(sentence):
                    if word not in frequencies:
                        frequencies[word] = 1
                    else:
                        frequencies[word] += 1

                    if frequencies[word] == self.freq_threshold and len(word)>1:
                        self.hi_stoi[word] = idx
                        self.hi_itos[idx] = word
                        idx += 1
    
    def english_vector(self, text):
        tokenized_text = self.tokenize_eng(text)
        
        numerical_sentence = [self.en_stoi['<sos>']]
        numerical_sentence += [self.en_stoi[word] if word in self.en_stoi else self.en_stoi['<unk>'] for word in tokenized_text]
        numerical_sentence.append(self.en_stoi['<eos>'])
        
        return numerical_sentence
    
    def hindi_vector(self, text):
        tokenized_text = self.tokenize_hin(text)
        
        numerical_sentence = [self.hi_stoi['<sos>']]
        numerical_sentence += [self.hi_stoi[word] if word in self.hi_stoi else self.hi_stoi['<unk>'] for word in tokenized_text]
        numerical_sentence.append(self.hi_stoi['<eos>'])
        
        return numerical_sentence
            

In [6]:
class TextData(Dataset):
    
    def __init__(self, freq_threshold, max_length):
                
        self.df = pd.read_csv('/kaggle/input/english-to-hindi/hin.txt',sep = '\t',names = ['english_sentence','hindi_sentence'])
        self.df.dropna(inplace = True)
        
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.df['english_sentence'],'english')
        self.vocab.build_vocabulary(self.df['hindi_sentence'],'hindi')
        
        print('English Vocabulary Size - ',len(self.vocab.en_stoi))
        print('Hindi Vocabulary Size - ',len(self.vocab.hi_stoi))
        
        self.vocab.en_stoi = dict(itertools.islice(self.vocab.en_stoi.items(), max_length))
        self.vocab.en_itos = dict(itertools.islice(self.vocab.en_itos.items(), max_length))
        
        self.vocab.hi_stoi = dict(itertools.islice(self.vocab.hi_stoi.items(), max_length))
        self.vocab.hi_itos = dict(itertools.islice(self.vocab.hi_itos.items(), max_length))
        
        self.pad_idx = self.vocab.en_stoi['<pad>']
    
        self.english_sentences = self.df['english_sentence'].to_list()
        self.hindi_sentences = self.df['hindi_sentence'].to_list()
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        eng_tokenized = torch.tensor(self.vocab.english_vector(self.english_sentences[index]))
        hin_tokenized = torch.tensor(self.vocab.hindi_vector(self.hindi_sentences[index]))
        return eng_tokenized, hin_tokenized

class MyCollate:
    
    def __init__(self,pad_idx):
        self.pad_idx = pad_idx
    
    def __call__(self, batch):
        eng_sentences = [item[0] for item in batch]
        hin_sentences = [item[1] for item in batch]
        
        eng_sentences = pad_sequence(eng_sentences, batch_first = False, padding_value = self.pad_idx)
        hin_sentences = pad_sequence(hin_sentences, batch_first = False, padding_value = self.pad_idx)
        
        return eng_sentences, hin_sentences

In [7]:
class Encoder(nn.Module):
    
    def __init__(self, input_size, embed_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        
        self.embed = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # shape of x: (seq_len, N) 
        # where N is batch size
        # seq_len is the number of words being sent
        embeddings = self.embed(x) 
        
        # embeddings shape: (seq_len, N, embed_size)
        outputs, (hidden,cell) = self.lstm(embeddings)
        
        # outputs shape: (seq_len ,N, hidden_size)
        return hidden, cell

class Decoder(nn.Module):
    
    def __init__(self, input_size, embed_size, hidden_size, output_size,num_layers):
        super(Decoder, self).__init__()
        
        self.embed = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x, hidden, cell):
        # Here x is of the shape (N), and we want to convert it to (1,N)
        # 1 here is seq_len, since we are passing only one word
        x = x.unsqueeze(0)
        
        # x shape: (1,N)
        embeddings = self.embed(x)
        
        # embeddings shape: (1,N,embed_size)
        outputs, (hidden, cell) = self.lstm(embeddings, (hidden, cell))
        
        # outputs shape: (1,N, hidden_size)
        predictions = self.fc(outputs)
        
        # predictions,shape: (1,N,output_size)
        # loss function needs shape (N, output_size) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell

class Translator(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Translator, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, source, target, teacher_force_ratio = 0.5):
        batch_size = source.shape[1] # shape: (seq_len, N)
        target_len = target.shape[0]
        target_vocab_size = len(dataset.vocab.en_stoi)
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        hidden,cell = self.encoder(source)
        
        x = target[0] # Get <sos>
        
        for t in range(1,target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            
            outputs[t] = output
            
            best_guess = torch.argmax(output)
            #print(best_guess)
            
            x = target[t]
        
        return outputs

In [33]:
dataset = TextData(10,260)

loader = DataLoader(
    dataset = dataset,
    batch_size = 32,
    shuffle = True,
    collate_fn = MyCollate(pad_idx = 0),
)

English Vocabulary Size -  265
Hindi Vocabulary Size -  368


In [37]:
num_epochs = 100
learning_rate = 0.001
batch_size = 32

input_size_encoder = len(dataset.vocab.en_stoi)
input_size_decoder = len(dataset.vocab.hi_stoi)

embed_size = 256

output_size = len(dataset.vocab.en_stoi)

hidden_size = 256
num_layers = 2

In [38]:
encoder = Encoder(input_size_encoder, embed_size, hidden_size, num_layers).to(device)
decoder = Decoder(input_size_decoder, embed_size, hidden_size, output_size, num_layers).to(device)
translator = Translator(encoder,decoder).to(device)

optimizer = optim.Adam(translator.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [39]:
for epoch in range(num_epochs):
    
    #translator.train()
    
    for batch_idx, (eng_sentences, hin_sentences) in tqdm(enumerate(loader),total = len(loader), leave = False):
        eng_sent_list = eng_sentences.to(device)
        hin_sent_list = hin_sentences.to(device)
        
        output = translator(eng_sent_list, hin_sent_list)
        
        output = output[1:].reshape(-1, output.shape[2])
        hin_sent_list = hin_sent_list[1:].reshape(-1)
        
        loss = criterion(output,hin_sent_list)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(translator.parameters(), max_norm=1)
        optimizer.step()
    
    print(f'Epoch - {epoch+1}/{num_epochs}, Loss - {loss}')

Epoch - 1/100, Loss - 3.4572300910949707


Epoch - 2/100, Loss - 2.9711129665374756


Epoch - 3/100, Loss - 2.851594924926758


Epoch - 4/100, Loss - 2.620299816131592


Epoch - 5/100, Loss - 2.4703550338745117


...

Epoch - 96/100, Loss - 0.031248953193426132


Epoch - 97/100, Loss - 0.04022153466939926


Epoch - 98/100, Loss - 0.04340507835149765


Epoch - 99/100, Loss - 0.02403227798640728


Epoch - 100/100, Loss - 0.05798852816224098


In [40]:
eng_sent = "What are you doing?"

sentence_tensor = torch.tensor(dataset.vocab.english_vector(eng_sent)).unsqueeze(1).to(device)

with torch.no_grad():
    hidden, cell = translator.encoder(sentence_tensor)


outputs = [dataset.vocab.hi_stoi['<sos>']]
for _ in range(100):
    previous_word = torch.tensor([outputs[-1]]).to(device)
    
    with torch.no_grad():
        output, hidden, cell = translator.decoder(previous_word, hidden, cell)
        best_guess = torch.argmax(output).item()
    
    outputs.append(best_guess)
    
    if output.argmax(1).item() == dataset.vocab.hi_stoi["<eos>"]:
            break

translated_sentence = [dataset.vocab.hi_itos[idx] for idx in outputs]

print(translated_sentence)

['<sos>', 'तुम', 'क्या', 'कर', 'रहे', '<unk>', '<eos>']
